In [1]:
# Block 1: Installation and Direct API Key Setup
# Install necessary libraries for interacting with the Gemini API, reading PDFs, and handling DOCX files.
!pip install -q -U google-generativeai pymupdf python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.0 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
import os
import shutil
import json
from google.colab import drive
import os
# For reading PDF files
import fitz  # PyMuPDF
# For reading DOCX files
import docx
import re

# --- API KEY CONFIGURATION ---
# WARNING: Pasting your API key directly in the code is a security risk.
# Avoid sharing this notebook or committing it to public version control.

# 1. Paste your Gemini API key in the line below.
API_KEY = "YOUR_GEMINI_API_KEY_HERE" # <-- PASTE YOUR KEY HERE

if API_KEY == "YOUR_GEMINI_API_KEY_HERE":
    print("🚨 PLEASE REPLACE 'YOUR_GEMINI_API_KEY_HERE' WITH YOUR ACTUAL API KEY.")
else:
    try:
        genai.configure(api_key=API_KEY)
        print("✅ Gemini API key configured successfully.")
    except Exception as e:
        print(f"🚨 An error occurred while configuring the API key: {e}")

# Initialize the Gemini Model
# We use gemini-1.5-flash for its speed and large context window, which is great for documents.
try:
    model = genai.GenerativeModel("gemini-1.5-flash")
    print(f"✅ Initialized Gemini Model: {model.model_name}")
except Exception as e:
    print(f"🚨 Failed to initialize the Gemini model. Please check if your API key is valid. Error: {e}")

✅ Gemini API key configured successfully.
✅ Initialized Gemini Model: models/gemini-1.5-flash


In [3]:
# Block 2: Mount Google Drive and Define Base Path

# # Mount your Google Drive. You'll be prompted to authorize this.
# try:
#     drive.mount('/content/drive')
#     print("✅ Google Drive mounted successfully.")
# except Exception as e:
#     print(f"🚨 Error mounting Google Drive: {e}")

# --- DEFINE YOUR BASE FOLDER PATH HERE ---
# This is the root folder containing all your raw resume files.
# Job-specific output folders will be created inside this directory.
APPLICANTS_FOLDER = '/content/drive/MyDrive/Applicants' # <-- CHANGE THIS PATH IF NEEDED

print(f"\nBase Applicants Folder set to: {APPLICANTS_FOLDER}")


Base Applicants Folder set to: /content/drive/MyDrive/Applicants


In [4]:
# Block 3: Configuration with Multiple Job Descriptions

# --- PIPELINE BEHAVIOR CONTROLS ---
Show_Explanation = True
Clean_Folder = True
SHORTLISTING_FLEXIBILITY = 'Flexible' # 'Strict', 'Balanced', or 'Flexible'

# --- COMPANY CONTEXT (Remains constant) ---
COMPANY_CONTEXT = """
Praxo, developed by MUDAR DZ LIMITED in Anantapur, India, is a dynamic mobile application that blends short-form video creation, social interaction, shopping, and creator monetization into a single platform.
Designed for both Android and iOS, the app allows users to create and share videos, engage in 1-to-1 and group chats, shop with cashback, and earn rewards through content engagement and scratch cards.
With features like live video marketing (“Face-to-Face Live”), kids mode, and location-based streaming, Praxo empowers creators to showcase their skills while earning royalties, cash, and gifts.
The platform reflects a growing focus on merging entertainment with e-commerce and community-driven monetization in the Indian digital ecosystem.
"""

# --- NEW: PROVIDE A LIST OF JOB DESCRIPTIONS ---
# The AI will find the best fit among these options for each candidate.
JOB_DESCRIPTIONS = [
    # Job Description 0
   """
**Job Title: Java Full Stack Developer**

**Location:** Ananthapur, Andhra Pradesh, India.

**Role Summary:**
We are seeking an enthusiastic and motivated Java Full Stack Developer (Entry-Level) to join our growing team.
This role is ideal for someone with foundational knowledge in backend technologies and a strong willingness to learn.
The candidate will work across backend systems, API integration, real-time communication, and cloud deployment.
While prior experience with tools like Next.js, Spring Boot, Node.js, Socket.IO, and AWS is preferred,
we are open to candidates who show potential and eagerness to grow—we will provide targeted training where needed.

**Key Responsibilities:**
- Collaborate with frontend and design teams to build, test, and deploy scalable backend services.
- Write and maintain clean, efficient server-side code using Node.js and/or Spring Boot.
- Develop RESTful and WebSocket-based APIs for mobile/web applications.
- Integrate real-time communication using Socket.IO.
- Assist in deploying and managing applications on AWS Cloud infrastructure.
- Implement basic backend control mechanisms (authentication, logging, error handling, etc.).
- Work with databases (SQL/NoSQL) to store and retrieve application data efficiently.
- Participate in code reviews, technical discussions, and sprint planning.
- Learn and adapt to new tools, frameworks, and best practices as needed for the project.

**Required Qualifications:**
- Bachelor's degree in Computer Science, Software Engineering, or a related field (or equivalent experience).
- Basic understanding of:
    Node.js or Spring Boot
    API development
    Real-time communication (e.g., WebSockets / Socket.IO)
    Next.js (good to have, especially for full-stack integrations)
    AWS basics (e.g., EC2, S3, or Lambda usage)
- Ability to read and understand existing codebases.
- Willingness to learn and take feedback positively.
- Strong problem-solving and communication skills.
- Familiarity with Git and collaborative development workflows.

**Nice to Have (but Not Mandatory)**
- Experience building and consuming APIs.
- Exposure to backend control systems like rate-limiting, caching, or logging tools.
- Projects or coursework involving real-time apps or microservices.

"""
# ,
#     # Job Description 1
#     """
# **Job Title: Andriod Developer**
# **Location:** Ananthapur, Andhra Pradesh, India.
# **Role Summary**
# We are looking for a passionate and curious Entry-Level Android Developer to join our mobile development team.
# This role is perfect for individuals with a basic grasp of Java, Kotlin, and Android application architecture.
# You’ll contribute to designing and maintaining mobile apps, fixing APIs, and integrating video playback functionality.
# We value candidates who are eager to learn and grow — and we’ll provide mentoring on the job where needed.

# **Key Responsibilities**
# Develop and maintain Android applications using Java and Kotlin.

# Collaborate with designers and backend teams to deliver seamless app functionality.

# Integrate and troubleshoot RESTful APIs within the app.

# Work on basic video player integration (e.g., ExoPlayer).

# Assist in refining the overall app design and performance.

# Write clean, modular, and well-documented code.

# Participate in code reviews, debugging, and sprint planning.

# **Requirements**
# Foundational experience with Java and Kotlin.

# Understanding of Android Studio and XML-based UI layouts.

# Familiarity with consuming REST APIs and error handling.

# Exposure to video player integration (ExoPlayer or MediaPlayer is a plus).

# Strong problem-solving and debugging skills.

# Willingness to learn and adapt to modern Android development practices.

# **Nice to Have**
# Personal or academic projects showcasing Android work.

# Knowledge of Jetpack components (ViewModel, LiveData, Navigation).

# Git or version control knowledge.
#     """,
#     # Job Description 2
#     """
# **Job Title: IOS Developer**
# **Location:** Ananthapur, Andhra Pradesh, India.
# **Role Summary**
# We are seeking a detail-oriented and driven Entry-Level iOS Developer to support our mobile application development efforts on Apple platforms. The ideal candidate has a foundation in Swift, SwiftUI, and Objective-C, with a basic understanding of app structure and video player integration. If you’re excited about building modern, intuitive iOS applications and open to learning as you go, we’d love to meet you.

# Key Responsibilities
# Contribute to iOS application development using Swift, SwiftUI, and occasionally Objective-C.

# Collaborate with cross-functional teams to define and implement features.

# Assist in integrating and debugging RESTful APIs.

# Support video playback features within the app (e.g., AVPlayer).

# Maintain application stability, performance, and responsiveness.

# Help with UI design implementation based on Figma or Sketch files.

# Participate in version control workflows and code documentation.

# **Requirements**
# Working knowledge of Swift and SwiftUI.

# Some exposure to Objective-C in academic or project settings.

# Familiarity with AVFoundation or similar for video playback (basic).

# Good understanding of Apple Human Interface Guidelines.

# Willingness to learn and grow within a collaborative environment.

# **Nice to Have**
# Experience with Xcode, Interface Builder, or Swift Packages.

# Basic understanding of iOS app lifecycle and MVC/MVVM patterns.

# Git or source control experience.
# """
]

print(f"✅ Configuration loaded with {len(JOB_DESCRIPTIONS)} job descriptions.")
print(f"ℹ️ Shortlisting Flexibility Level set to: '{SHORTLISTING_FLEXIBILITY}'")

✅ Configuration loaded with 1 job descriptions.
ℹ️ Shortlisting Flexibility Level set to: 'Flexible'


In [5]:
# Block 4: Dynamic Path Generation for Multiple Jobs

def extract_and_sanitize_job_title(description):
    """Extracts the job title and creates a folder-friendly string."""
    match = re.search(r"Job Title:\s*(.*)", description)
    if match:
        title = match.group(1).strip()
        sanitized_title = re.sub(r'[^a-zA-Z0-9\s]', '', title).lower().replace(' ', '_')
        return sanitized_title
    return "general_application"

# Store paths for each job
job_output_folders = []
for jd in JOB_DESCRIPTIONS:
    folder_name = extract_and_sanitize_job_title(jd)
    path = os.path.join(APPLICANTS_FOLDER, folder_name, 'shortlisted')
    job_output_folders.append(path)

# Define top-level folders for candidates who don't fit any role or cause errors
NOT_SHORTLISTED_FOLDER = os.path.join(APPLICANTS_FOLDER, 'not_shortlisted_for_any_role')
ERRORS_FOLDER = os.path.join(APPLICANTS_FOLDER, 'errors')

# Create all necessary directories
print("--- Dynamic Folder Setup ---")
for path in job_output_folders:
    os.makedirs(path, exist_ok=True)
    print(f"Created: {path}")

os.makedirs(NOT_SHORTLISTED_FOLDER, exist_ok=True)
print(f"Created: {NOT_SHORTLISTED_FOLDER}")
os.makedirs(ERRORS_FOLDER, exist_ok=True)
print(f"Created: {ERRORS_FOLDER}")

print("\n✅ Multi-job output folders are ready.")

--- Dynamic Folder Setup ---
Created: /content/drive/MyDrive/Applicants/java_full_stack_developer/shortlisted
Created: /content/drive/MyDrive/Applicants/not_shortlisted_for_any_role
Created: /content/drive/MyDrive/Applicants/errors

✅ Multi-job output folders are ready.


In [6]:
# Block 5: Helper Functions (With Rate Limit Handling)

import time # Import the time module to handle delays

def clean_output_folders():
    # ... (This function is unchanged)
    print("Cleaning all output folders...")
    for path in job_output_folders:
        folder_to_clean = os.path.dirname(path)
        if os.path.isdir(folder_to_clean): shutil.rmtree(folder_to_clean)
    for folder in [NOT_SHORTLISTED_FOLDER, ERRORS_FOLDER]:
        if os.path.isdir(folder): shutil.rmtree(folder)
    print("✅ All previous job folders and output folders cleared.")

def get_resume_text(file_path):
    # ... (This function is unchanged)
    text = ""
    try:
        if file_path.lower().endswith(".pdf"):
            with fitz.open(file_path) as doc:
                for page in doc: text += page.get_text()
        elif file_path.lower().endswith(".docx"):
            doc = docx.Document(file_path)
            for para in doc.paragraphs: text += para.text + "\n"
        else: return None, "Unsupported file format."
        if not text.strip(): return None, "File is empty."
        return text, None
    except Exception as e: return None, f"Error reading file: {e}"

def get_unique_destination_path(dest_folder, filename):
    # ... (This function is unchanged)
    base, ext = os.path.splitext(filename)
    counter = 2
    new_filename = filename
    dest_path = os.path.join(dest_folder, new_filename)
    while os.path.exists(dest_path):
        new_filename = f"{base}_{counter}{ext}"
        dest_path = os.path.join(dest_folder, new_filename)
        counter += 1
    return dest_path

def get_flexibility_instructions(level):
    # ... (This function is unchanged)
    if level == 'Strict':
        return "Be very strict. The candidate MUST meet almost all 'Required Qualifications' to be a fit."
    elif level == 'Flexible':
        return "Be flexible. Look for high-potential candidates with strong projects or transferable skills, even if they don't meet all formal requirements."
    else: # Balanced
        return "Be balanced. A candidate is a fit if they meet most core requirements or compensate for a missing requirement with strong, relevant project work or certifications."

def check_fit_for_single_job_with_backoff(resume_text, job_description, flexibility_level):
    """
    (ROBUST API CALL) Analyzes a resume against a single job with rate limit handling.
    This function will automatically wait and retry if it hits a rate limit error.
    """
    flexibility_instructions = get_flexibility_instructions(flexibility_level)
    prompt = f"""
    You are an expert HR agent performing a screening check.
    Your task is to decide if a candidate is a suitable fit for this one specific job.
    **Company Context:** {COMPANY_CONTEXT}
    **Candidate's Resume Text:** {resume_text}
    **Job Description to Check Against:** {job_description}
    **Your Task & Decision Rule:**
    - {flexibility_instructions}
    - Based on this rule, is the candidate a good fit for this specific job?
    **Respond ONLY with a valid JSON object in the format below:**
    {{
      "decision": "shortlist" or "not_shortlisted",
      "reason": "A concise, one-sentence explanation for your decision."
    }}
    """

    # Exponential backoff parameters
    max_retries = 5
    base_delay_seconds = 2

    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            cleaned_response = response.text.strip().lstrip('```json').rstrip('```').strip()
            result = json.loads(cleaned_response)
            return result, None # Success
        except Exception as e:
            # Check if the error is a rate limit error (often contains '429')
            if "429" in str(e):
                if attempt < max_retries - 1:
                    wait_time = base_delay_seconds * (2 ** attempt)
                    print(f"    -> Rate limit hit. Waiting for {wait_time} seconds before retrying...")
                    time.sleep(wait_time)
                else:
                    print(f"    -> Rate limit error persisted after {max_retries} retries. Giving up on this API call.")
                    return None, f"API rate limit error after multiple retries: {e}"
            else:
                # It's a different error (e.g., JSON parsing), so don't retry.
                raw_response = response.text if 'response' in locals() else 'N/A'
                return None, f"Gemini API call or JSON parsing failed: {e}. Raw response: {raw_response}"

print("✅ Helper functions updated with robust rate limit handling.")

✅ Helper functions updated with robust rate limit handling.


In [7]:
# Block 6: Main Sequential-Matching Pipeline (Updated)

if Clean_Folder:
    clean_output_folders()
    # Re-create folders after cleaning
    for path in job_output_folders: os.makedirs(path, exist_ok=True)
    os.makedirs(NOT_SHORTLISTED_FOLDER, exist_ok=True)
    os.makedirs(ERRORS_FOLDER, exist_ok=True)

print(f"\n🚀 Starting sequential role matching using '{SHORTLISTING_FLEXIBILITY}' flexibility...")

if not os.path.isdir(APPLICANTS_FOLDER) or len(os.listdir(APPLICANTS_FOLDER)) == 0:
    print(f"🚨 The applicants folder is empty or does not exist: {APPLICANTS_FOLDER}")
else:
    for filename in os.listdir(APPLICANTS_FOLDER):
        source_path = os.path.join(APPLICANTS_FOLDER, filename)
        if not os.path.isfile(source_path): continue

        print(f"\nProcessing resume: {filename}")
        resume_text, error = get_resume_text(source_path)
        if error:
            print(f"  -> ERROR: {error}")
            shutil.copy(source_path, get_unique_destination_path(ERRORS_FOLDER, filename))
            continue

        match_found = False
        final_destination_folder = None

        # Sequentially check against each job description
        for i, jd in enumerate(JOB_DESCRIPTIONS):
            job_title = extract_and_sanitize_job_title(jd)
            print(f"  -> Checking against Job #{i+1}: '{job_title}'...")

            # *** THIS IS THE ONLY LINE THAT CHANGES IN THIS BLOCK ***
            analysis, error = check_fit_for_single_job_with_backoff(resume_text, jd, SHORTLISTING_FLEXIBILITY)

            if error:
                # Now, this error is more serious as retries have already failed
                print(f"    -> CRITICAL ERROR during analysis: {error}")
                shutil.copy(source_path, get_unique_destination_path(ERRORS_FOLDER, filename))
                match_found = True # Mark as 'processed' to prevent it from going to not_shortlisted
                break # Stop processing this file and move to the next

            decision = analysis.get("decision", "not_shortlisted").lower()
            reason = analysis.get("reason", "No reason provided.")

            if decision == "shortlist":
                print(f"  ✅ Match Found for '{job_title}'!")
                if Show_Explanation: print(f"     Reason: {reason}")
                final_destination_folder = job_output_folders[i]
                match_found = True
                break
            else:
                 if Show_Explanation: print(f"    -> Not a fit. Reason: {reason}")


        if not match_found:
            print(f"  -> No suitable role found for {filename} after checking all jobs.")
            final_destination_folder = NOT_SHORTLISTED_FOLDER

        if final_destination_folder:
            dest_path = get_unique_destination_path(final_destination_folder, filename)
            shutil.copy(source_path, dest_path)
            print(f"  -> Copied to '{os.path.relpath(final_destination_folder, APPLICANTS_FOLDER)}' folder.")

print("\n\n🎉 Pipeline finished. All resumes have been checked and sorted.")

Cleaning all output folders...
✅ All previous job folders and output folders cleared.

🚀 Starting sequential role matching using 'Flexible' flexibility...

Processing resume: Pranav_Mudar.pdf
  -> Checking against Job #1: 'java_full_stack_developer'...
  ✅ Match Found for 'java_full_stack_developer'!
     Reason: Pranav's diverse project portfolio, including experience with socket programming and database management, demonstrates potential and transferable skills relevant to the Java Full Stack Developer role despite lacking direct experience with Spring Boot or Node.js.
  -> Copied to 'java_full_stack_developer/shortlisted' folder.

Processing resume: Tharun_Neela.pdf
  -> Checking against Job #1: 'java_full_stack_developer'...
    -> Not a fit. Reason: The candidate's skills and experience are primarily focused on frontend development and embedded systems, lacking the required Java, Spring Boot, Node.js, or relevant backend expertise for the Java Full Stack Developer role.
  -> No su

In [8]:
# Block 7: Detailed Analysis of a Matched Candidate

# --- INPUT THE FILENAME OF THE CANDIDATE YOU WANT TO ANALYZE ---
candidate_filename = "pranav.pdf" # <-- CHANGE THIS FILENAME

def get_detailed_report_for_match(filename):
    """(API CALL 2) Finds a candidate and generates a detailed report against the job they were matched with."""
    file_path, found_in_folder, job_index = None, None, -1

    # Search in all job-specific shortlisted folders
    for i, folder_path in enumerate(job_output_folders):
        path_to_check = os.path.join(folder_path, filename)
        if os.path.exists(path_to_check):
            file_path, found_in_folder, job_index = path_to_check, folder_path, i
            break

    # If not found, search in not_shortlisted and errors
    if not file_path:
        for folder in [NOT_SHORTLISTED_FOLDER, ERRORS_FOLDER]:
            path_to_check = os.path.join(folder, filename)
            if os.path.exists(path_to_check):
                file_path, found_in_folder = path_to_check, folder
                break

    if not file_path:
        print(f"🚨 Could not find the file '{filename}' in any of the output folders.")
        return

    # Determine the context for the final report
    if job_index != -1: # Candidate was shortlisted for a specific job
        job_title = os.path.basename(os.path.dirname(found_in_folder))
        relevant_jd = JOB_DESCRIPTIONS[job_index]
        analysis_context = f"The candidate was shortlisted for the '{job_title}' role."
    else: # Candidate was not shortlisted or errored
        relevant_jd = "\n".join([f"--- JOB {i} ---\n{jd}" for i, jd in enumerate(JOB_DESCRIPTIONS)])
        analysis_context = f"The candidate was placed in the '{os.path.basename(found_in_folder)}' folder and not matched to a specific role."

    resume_text, error = get_resume_text(file_path)
    if error:
        print(f"🚨 Could not read the file. Reason: {error}")
        return

    print(f"\nGenerating detailed analysis for '{filename}'...")
    print(f"Context: {analysis_context}")

    detailed_prompt = f"""
    You are a senior hiring manager providing a final analysis. {analysis_context}
    The initial screening was done with a '{SHORTLISTING_FLEXIBILITY}' flexibility level.
    Your analysis must be objective and based strictly on the resume against the relevant job description(s).

    **Relevant Job Description(s):**
    {relevant_jd}

    **Candidate's Resume Text:**
    {resume_text}

    **Your Analysis Task:**
    Provide a detailed report with the following markdown sections:
    ### Overall Summary
    A summary of the candidate's fit. If they were shortlisted, explain why they match the specific role. If not, explain the general mismatch against the available roles.
    ### Strengths Aligned with Role
    - List 3-4 key strengths aligned with the specific job they were matched with (or general strengths if not matched).
    ### Potential Gaps or Areas to Clarify
    - List weaknesses or areas to probe in an interview.
    """
    try:
        response = model.generate_content(detailed_prompt)
        from IPython.display import display, Markdown
        print("\n" + "="*40 + "\n    DETAILED CANDIDATE REPORT\n" + "="*40 + "\n")
        display(Markdown(response.text))
        print("\n" + "="*40)
    except Exception as e:
        print(f"🚨 An error occurred during detailed analysis: {e}")


# --- Run the detailed analysis ---
if 'candidate_filename' in locals() and candidate_filename and candidate_filename != "pranav.pdf":
    get_detailed_report_for_match(candidate_filename)
else:
    print("ℹ️ Please set the 'candidate_filename' variable in this block to a processed resume file and re-run this cell.")

ℹ️ Please set the 'candidate_filename' variable in this block to a processed resume file and re-run this cell.
